# 📊 Football Player Analytics Pipeline
## Notebook 3b: Hierarchical Clustering

This notebook explores **Hierarchical Clustering** as an alternative to K-Means.

**Key advantages of Hierarchical Clustering:**
- Creates a tree-like structure (dendrogram) showing how players relate
- No need to specify k upfront
- Reveals nested groupings (sub-clusters within clusters)
- Better interpretability of cluster relationships

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import pdist

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

DATA_DIR = Path("../data")
PROCESSED_DIR = DATA_DIR / "processed"
OUTPUT_DIR = Path("../outputs")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Libraries loaded!")

## 1. Load Data

In [ ]:
# Load processed forwards
data_file = PROCESSED_DIR / "forwards_processed.csv"

if not data_file.exists():
    raise FileNotFoundError(f"Run Notebook 02 first! Missing: {data_file}")

df = pd.read_csv(data_file)
print(f"✅ Loaded {len(df)} forwards")
print(f"\n📋 Available columns: {len(df.columns)}")

In [ ]:
# Identify key columns (case-insensitive)
def find_col(df, names):
    """Find column by name, case-insensitive."""
    for name in names:
        for col in df.columns:
            if col.lower() == name.lower():
                return col
    return None

PLAYER_COL = find_col(df, ['Player', 'player', 'Name', 'name'])
TEAM_COL = find_col(df, ['Squad', 'squad', 'Team', 'team'])
LEAGUE_COL = find_col(df, ['League', 'league', 'Comp', 'comp'])

print(f"Player column: {PLAYER_COL}")
print(f"Team column: {TEAM_COL}")
print(f"League column: {LEAGUE_COL}")

In [ ]:
# Load or find clustering features
feature_file = PROCESSED_DIR / "clustering_features.txt"
if feature_file.exists():
    with open(feature_file, 'r') as f:
        CLUSTERING_FEATURES = [line.strip() for line in f.readlines() if line.strip()]
else:
    # Find all per90 columns that aren't normalized versions
    CLUSTERING_FEATURES = [c for c in df.columns if 'per90' in c.lower() and '_norm' not in c]

print(f"\n📊 Using {len(CLUSTERING_FEATURES)} features for clustering:")
for i, f in enumerate(CLUSTERING_FEATURES, 1):
    print(f"  {i}. {f}")

## 2. Prepare Data

In [ ]:
# Create feature matrix
X = df[CLUSTERING_FEATURES].fillna(0).values

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print(f"📊 Feature matrix: {X_scaled.shape[0]} players × {X_scaled.shape[1]} features")

## 3. Compute Hierarchical Clustering

We'll try different **linkage methods**:
- `ward`: Minimizes variance within clusters (most common)
- `average`: Uses average distance between all pairs
- `complete`: Uses maximum distance between clusters
- `single`: Uses minimum distance between clusters

In [ ]:
# Compute linkage matrices for different methods
linkage_methods = ['ward', 'average', 'complete']
linkage_results = {}

for method in linkage_methods:
    print(f"Computing {method} linkage...")
    if method == 'ward':
        Z = linkage(X_scaled, method=method)
    else:
        # For non-ward methods, we can use different distance metrics
        Z = linkage(X_scaled, method=method, metric='euclidean')
    linkage_results[method] = Z

print("\n✅ All linkage matrices computed!")

## 4. Visualize Dendrograms

A dendrogram shows how clusters are formed at each level.

In [ ]:
# Plot dendrograms for each method
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, method in zip(axes, linkage_methods):
    dendrogram(
        linkage_results[method],
        truncate_mode='lastp',  # Show only last p merged clusters
        p=30,  # Show 30 clusters
        leaf_rotation=90,
        leaf_font_size=8,
        ax=ax
    )
    ax.set_title(f'{method.capitalize()} Linkage', fontsize=14, fontweight='bold')
    ax.set_xlabel('Cluster Size')
    ax.set_ylabel('Distance')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'hierarchical_dendrograms.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Full dendrogram with Ward linkage (zoomed out view)
plt.figure(figsize=(16, 8))

dendrogram(
    linkage_results['ward'],
    truncate_mode='level',
    p=5,  # Show 5 levels
    show_contracted=True,
    leaf_rotation=90,
    leaf_font_size=8
)

plt.title('Ward Linkage Dendrogram (Truncated)', fontsize=14, fontweight='bold')
plt.xlabel('Sample Index or (Cluster Size)')
plt.ylabel('Distance')

# Add horizontal lines for potential cuts
plt.axhline(y=40, color='r', linestyle='--', alpha=0.5, label='Potential cut (k=5-7)')
plt.axhline(y=25, color='orange', linestyle='--', alpha=0.5, label='Potential cut (k=8-10)')
plt.legend()

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'hierarchical_dendrogram_full.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Choose Optimal Number of Clusters

We'll use silhouette score to find the best cut.

In [ ]:
# Test different numbers of clusters
K_RANGE = range(4, 13)
results = []

# Using Ward linkage (generally best for compact clusters)
Z_ward = linkage_results['ward']

print("Testing cluster counts...\n")
for k in K_RANGE:
    labels = fcluster(Z_ward, k, criterion='maxclust')
    sil = silhouette_score(X_scaled, labels)
    results.append({'k': k, 'silhouette': sil, 'method': 'ward'})
    print(f"  k={k:2d}: silhouette={sil:.3f}")

results_df = pd.DataFrame(results)
best_k = int(results_df.loc[results_df['silhouette'].idxmax(), 'k'])
print(f"\n✅ Best k = {best_k}")

In [ ]:
# Compare linkage methods across different k values
comparison_results = []

for method in linkage_methods:
    Z = linkage_results[method]
    for k in K_RANGE:
        labels = fcluster(Z, k, criterion='maxclust')
        sil = silhouette_score(X_scaled, labels)
        comparison_results.append({'k': k, 'silhouette': sil, 'method': method})

comparison_df = pd.DataFrame(comparison_results)

# Plot comparison
fig, ax = plt.subplots(figsize=(10, 5))
for method in linkage_methods:
    data = comparison_df[comparison_df['method'] == method]
    ax.plot(data['k'], data['silhouette'], '-o', label=method.capitalize())

ax.axvline(x=best_k, color='gray', linestyle='--', alpha=0.5, label=f'Best k={best_k}')
ax.set_xlabel('Number of Clusters (k)')
ax.set_ylabel('Silhouette Score')
ax.set_title('Linkage Method Comparison')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'hierarchical_method_comparison.png', dpi=150)
plt.show()

In [ ]:
# Choose number of clusters (you can adjust this)
N_CLUSTERS = best_k  # Use the best k found, or manually set
LINKAGE_METHOD = 'ward'  # Best for balanced, compact clusters

print(f"\n🎯 Using {N_CLUSTERS} clusters with {LINKAGE_METHOD} linkage")

## 6. Apply Final Clustering

In [ ]:
# Get cluster assignments
Z_final = linkage_results[LINKAGE_METHOD]
df['cluster'] = fcluster(Z_final, N_CLUSTERS, criterion='maxclust') - 1  # Zero-indexed

print("✅ Hierarchical clustering complete!")
print(f"\nPlayers per cluster:")
print(df['cluster'].value_counts().sort_index())

## 7. Describe Each Cluster

Same approach as K-Means: describe based on statistical profiles.

In [ ]:
# Calculate z-scores for each cluster
overall_mean = df[CLUSTERING_FEATURES].mean()
overall_std = df[CLUSTERING_FEATURES].std()

cluster_profiles = df.groupby('cluster')[CLUSTERING_FEATURES].mean()
cluster_zscores = (cluster_profiles - overall_mean) / overall_std

print("📊 Cluster Z-Scores (green=high, red=low):")
styled = cluster_zscores.style.background_gradient(cmap='RdYlGn', axis=None, vmin=-2, vmax=2)
display(styled)

In [ ]:
# Create descriptive labels
def describe_cluster(z_scores: pd.Series) -> str:
    """
    Create a description based on what the cluster is high/low in.
    No fancy names - just factual descriptions.
    """
    high_traits = z_scores[z_scores > 0.5].sort_values(ascending=False)
    low_traits = z_scores[z_scores < -0.5].sort_values()
    
    parts = []
    
    if len(high_traits) > 0:
        high_names = [t.replace('_per90', '').replace('_', ' ') for t in high_traits.index[:3]]
        parts.append(f"HIGH: {', '.join(high_names)}")
    
    if len(low_traits) > 0:
        low_names = [t.replace('_per90', '').replace('_', ' ') for t in low_traits.index[:2]]
        parts.append(f"LOW: {', '.join(low_names)}")
    
    if not parts:
        return "Average across all metrics"
    
    return " | ".join(parts)

# Describe each cluster
cluster_descriptions = {}
print("\n" + "="*80)
print("CLUSTER DESCRIPTIONS (Hierarchical - based on statistical analysis)")
print("="*80)

for cluster_id in range(N_CLUSTERS):
    z_scores = cluster_zscores.loc[cluster_id]
    description = describe_cluster(z_scores)
    count = (df['cluster'] == cluster_id).sum()
    
    cluster_descriptions[cluster_id] = f"Cluster {cluster_id}: {description}"
    
    print(f"\nCluster {cluster_id} ({count} players)")
    print("-" * 60)
    print(description)
    
    # Show key metrics
    print("\nKey metrics:")
    for feat in CLUSTERING_FEATURES:
        z = z_scores[feat]
        val = cluster_profiles.loc[cluster_id, feat]
        if abs(z) > 0.3:
            direction = "↑" if z > 0 else "↓"
            print(f"  {direction} {feat}: {val:.3f} (z={z:+.2f})")

# Add descriptions to dataframe
df['cluster_name'] = df['cluster'].map(cluster_descriptions)

In [ ]:
# Heatmap of cluster profiles
fig, ax = plt.subplots(figsize=(14, 8))

# Clean column names for display
display_cols = [c.replace('_per90', '').replace('_', ' ').title() for c in CLUSTERING_FEATURES]
plot_data = cluster_zscores.copy()
plot_data.columns = display_cols
plot_data.index = [f"Cluster {i}" for i in plot_data.index]

sns.heatmap(plot_data, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
            vmin=-2, vmax=2, ax=ax, cbar_kws={'label': 'Z-Score'})

ax.set_title('Hierarchical Cluster Profiles (Z-Score: how different from average)', fontsize=14, fontweight='bold')
ax.set_xlabel('Features')
ax.set_ylabel('Cluster')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'hierarchical_cluster_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Visualize Clusters (2D PCA)

In [ ]:
# PCA for 2D visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
df['pca_1'] = X_pca[:, 0]
df['pca_2'] = X_pca[:, 1]

print(f"PCA explains {pca.explained_variance_ratio_.sum()*100:.1f}% of variance")

In [ ]:
# Scatter plot
fig, ax = plt.subplots(figsize=(12, 8))

colors = plt.cm.tab10(np.linspace(0, 1, N_CLUSTERS))

for cluster_id in range(N_CLUSTERS):
    mask = df['cluster'] == cluster_id
    count = mask.sum()
    ax.scatter(df.loc[mask, 'pca_1'], df.loc[mask, 'pca_2'], 
               color=colors[cluster_id],
               label=f'Cluster {cluster_id} (n={count})', alpha=0.6, s=30)

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
ax.set_title('Hierarchical Clustering (PCA Projection)')
ax.legend(bbox_to_anchor=(1.02, 1), loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'hierarchical_cluster_scatter.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Compare with K-Means (if available)

In [ ]:
# Load K-Means results if available
kmeans_file = PROCESSED_DIR / "forwards_clustered.csv"

if kmeans_file.exists():
    df_kmeans = pd.read_csv(kmeans_file)
    
    # Merge on player
    if PLAYER_COL and PLAYER_COL in df_kmeans.columns:
        df_compare = df[[PLAYER_COL, 'cluster']].merge(
            df_kmeans[[PLAYER_COL, 'cluster']],
            on=PLAYER_COL,
            suffixes=('_hierarchical', '_kmeans')
        )
        
        # Cross-tabulation
        cross_tab = pd.crosstab(
            df_compare['cluster_hierarchical'], 
            df_compare['cluster_kmeans'],
            margins=True
        )
        
        print("📊 Hierarchical vs K-Means Cluster Comparison:")
        print("\nCross-tabulation (rows=Hierarchical, cols=K-Means):")
        display(cross_tab)
        
        # Calculate agreement
        from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
        ari = adjusted_rand_score(df_compare['cluster_hierarchical'], df_compare['cluster_kmeans'])
        nmi = normalized_mutual_info_score(df_compare['cluster_hierarchical'], df_compare['cluster_kmeans'])
        
        print(f"\n📈 Clustering Agreement:")
        print(f"  Adjusted Rand Index: {ari:.3f} (1.0 = perfect agreement)")
        print(f"  Normalized Mutual Info: {nmi:.3f} (1.0 = perfect agreement)")
else:
    print("ℹ️ No K-Means results found. Run notebook 03_clustering.ipynb first for comparison.")

## 10. Sample Players by Cluster

In [ ]:
# Show sample players from each cluster
print("\n" + "="*80)
print("SAMPLE PLAYERS BY CLUSTER (Hierarchical)")
print("="*80)

for cluster_id in range(N_CLUSTERS):
    cluster_df = df[df['cluster'] == cluster_id]
    print(f"\nCluster {cluster_id} ({len(cluster_df)} players)")
    print("-" * 40)
    
    # Sort by a key metric and show top 5
    sort_col = 'goals_per90' if 'goals_per90' in df.columns else CLUSTERING_FEATURES[0]
    top = cluster_df.nlargest(5, sort_col)
    
    for _, row in top.iterrows():
        player = row[PLAYER_COL] if PLAYER_COL else 'Unknown'
        team = row[TEAM_COL] if TEAM_COL else ''
        league = row[LEAGUE_COL] if LEAGUE_COL else ''
        if league:
            print(f"  • {player} ({team}, {league})")
        else:
            print(f"  • {player} ({team})")

## 11. Save Results

In [ ]:
# Save clustered data
output_file = PROCESSED_DIR / "forwards_hierarchical_clustered.csv"
df.to_csv(output_file, index=False)
print(f"💾 Saved: {output_file}")

# Save model and metadata
model_data = {
    'linkage_matrix': Z_final,
    'linkage_method': LINKAGE_METHOD,
    'scaler': scaler,
    'pca': pca,
    'features': CLUSTERING_FEATURES,
    'cluster_names': cluster_descriptions,
    'cluster_profiles': cluster_profiles,
    'cluster_zscores': cluster_zscores,
    'n_clusters': N_CLUSTERS
}

with open(OUTPUT_DIR / "hierarchical_clustering_model.pkl", 'wb') as f:
    pickle.dump(model_data, f)
print(f"💾 Saved: hierarchical_clustering_model.pkl")

---
## ✅ Done!

This notebook demonstrated **Hierarchical Clustering** as an alternative to K-Means.

**Key takeaways:**
- Dendrograms reveal the nested structure of player relationships
- Ward linkage typically produces the most balanced clusters
- Results can be compared with K-Means using Adjusted Rand Index

**Next steps:**
- Try different linkage methods (average, complete) for different cluster shapes
- Experiment with distance metrics (cosine, Manhattan)
- Use the dendrogram to identify sub-clusters within clusters